In [1]:
import numpy as np
import pandas as pd
import pickle
import os

from tqdm import tqdm
from generate_multivariate_samples import generate_multivariate_samples
from sklearn.model_selection import train_test_split

import numpy as np
import plotly.graph_objs as go

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge


In [2]:
df_all = pd.read_csv('eua_price_data.csv', thousands=',') 
df_all['Date'] = pd.to_datetime(df_all['Date'], format='%Y-%m-%d')  
df_all = df_all.sort_values(by = 'Date', ascending=True).reset_index(drop = True)
df_all = df_all[(df_all['Date'] > pd.to_datetime('2020-11-24')) & (df_all['Date'] < pd.to_datetime('2024-10-07'))].reset_index(drop=True)

In [3]:
# import plotly.express as px
# # Assuming df_all is your DataFrame
# fig = px.line(df_all, x='Date', y='EUA', title='EUA Over Time')
# # Show the figure
# fig.show()

In [4]:
from curate_training_test_data import curate_training_test_data

In [5]:
sklearn_models = {
    'rf':{'model': RandomForestRegressor(),
          'param_grid': {
          'n_estimators': [50, 100, 200],            # Number of trees in the forest
          'max_depth': [None, 10, 20, 30],           # Maximum depth of the tree
          'min_samples_split': [2, 5, 10],           # Minimum number of samples required to split an internal node
          'min_samples_leaf': [1, 2, 4],             # Minimum number of samples required to be at a leaf node
          'bootstrap': [True, False]                 # Whether bootstrap samples are used when building trees
                        }},
    'knn':{'model': KNeighborsRegressor(),
          'param_grid': {
    'n_neighbors': [3, 5, 7, 9],      # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weight function
    'p': [1, 2]                        # Power parameter for the Minkowski metric (1 = Manhattan, 2 = Euclidean)
}},



#     'dt':{'model': DecisionTreeRegressor(),
#             'param_grid': {
#     'max_depth': [None, 10, 20, 30],              # Maximum depth of the tree
#     'min_samples_split': [2, 5, 10],              # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],                # Minimum number of samples required to be at a leaf node
#     'max_features': [None, 'auto', 'sqrt', 'log2']  # Number of features to consider when looking for the best split
# }},



#     'lasso':{'model': Lasso(),
#             'param_grid': {
#     'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],  # Regularization strength
#     'fit_intercept': [True, False],          # Whether to calculate the intercept
#     'max_iter': [1000, 2000, 3000]           # Maximum number of iterations
# }},


#     'ridge':{'model': Ridge(),
#             'param_grid':{
#     'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization strength
#     'fit_intercept': [True, False],                 # Whether to calculate the intercept
#     'solver': ['auto', 'svd', 'cholesky', 'sparse_cg', 'saga'],  # Solver to use for optimization
#     'max_iter': [1000, 2000, 3000]                  # Maximum number of iterations
# }},


#     'gb':{'model': GradientBoostingRegressor(),
#             'param_grid': {
#     'n_estimators': [100, 200, 300],          # Number of boosting stages
#     'learning_rate': [0.01, 0.1, 0.2],        # Learning rate
#     'max_depth': [3, 4, 5],                   # Maximum depth of individual trees
#     'subsample': [0.8, 1.0],                  # Fraction of samples used for fitting each base learner
#     'min_samples_split': [2, 5, 10]           # Minimum number of samples required to split a node
# }},

}

In [6]:
folder_name = 'sklearn_models_v2'
test_date= '2024-07-01'

original_EUA = df_all['EUA'].values  
dates = df_all['Date'].values
for key, values in sklearn_models.items():
    
    modeltype = key
    for sequence_length in [14]:
        predictors_lst = ['EUA', 'Oil', 'Coal', 'NG', 'USEU', 'S&P_clean', 'DAX']

        X_train, y_train, X_test, y_test, scaler = curate_training_test_data(df_all, 
                                                                            sequence_length=sequence_length,
                                                                            test_date = test_date,
                                                                            predictors_lst = predictors_lst )

        if os.path.isfile(f'{folder_name}/{modeltype}_bestmodel_ts_{sequence_length}.pkl'):
            continue
        
        # Initialize the model
        model = values['model']

        # Define the parameter grid to search for the best hyperparameters
        param_grid = values['param_grid']

        # Set up the GridSearchCV
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                                cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

        # Fit the model with grid search to the training data
        grid_search.fit(X_train, y_train)

        # Get the best parameters and model from the grid search
        best_params = grid_search.best_params_
        best_model = grid_search.best_estimator_

        # Print the best parameters
        print(f"Best parameters found: {best_params}")

        # accuracy plot
        train_predictions = best_model.predict(X_train);
        train_predictions_rescaled = scaler.inverse_transform(train_predictions)
        test_predictions = best_model.predict(X_test);
        test_predictions_rescaled = scaler.inverse_transform(test_predictions)

        ground_truth_train = scaler.inverse_transform(y_train)
        ground_truth_test = scaler.inverse_transform(y_test)

        plt.figure(figsize = (13,13))
        for i, feature in enumerate(predictors_lst):
            plt.subplot(len(predictors_lst)//3 + 1 if len(predictors_lst)%3 !=0 else len(predictors_lst)//3, 3, i+1)
            plt.scatter(ground_truth_train[:,i],train_predictions_rescaled[:,i], label = 'train')
            plt.scatter(ground_truth_test[:,i],test_predictions_rescaled[:,i], label = 'test')
            plt.plot([min(ground_truth_train[:,i]), max(ground_truth_train[:,i])], 
                    [min(ground_truth_train[:,i]), max(ground_truth_train[:,i])], color='red', label='1:1 Line')
            
            r2_train = r2_score(ground_truth_train[:,i],train_predictions_rescaled[:,i])
            r2_test = r2_score(ground_truth_train[:,i],train_predictions_rescaled[:,i])
            plt.title(f"{feature} - train: {r2_train:.5f} / test: {r2_test:.5f}")
            plt.legend()
            plt.grid('on')
            plt.xlabel('ground truth')
            plt.ylabel('prediction')
        plt.tight_layout()
        plt.savefig(f'{folder_name}/{modeltype}_acc_ts_{sequence_length}.pdf')

        # get RMSE:
        rel_erorrs = []
        for i in range(test_predictions_rescaled.shape[1]):
            prediction = test_predictions[:, i]
            ground_truth = y_test[:,i]
            rel_error = np.mean(np.sqrt(((prediction-ground_truth)**2)))
            print(predictors_lst[i])
            print(rel_error)
            rel_erorrs.append(rel_error)


        num_of_prediction = 30*24
        num_ensemble = 100
        rel_erorrs_mat = np.array([rel_erorrs for i in range(num_ensemble)])
        corr = df_all[predictors_lst].corr()
        for factor in [0.5, 1.0, 1.5, 2.0, 3.0, 5.0]:
            next_predictions = []
            current_input = X[-1].flatten().reshape(1,-1)
            current_input = np.array([current_input for i in range(num_ensemble)]).squeeze()
            for iter_ in tqdm(range(num_of_prediction)):
                next_prediction = best_model.predict(current_input)
                error_p = generate_multivariate_samples(corr, n_samples=num_ensemble)
                next_prediction = next_prediction * (1+error_p*rel_erorrs_mat*factor)
                next_predictions.append(next_prediction)
                current_input = np.concatenate([current_input[:,len(predictors_lst):], 
                                                next_prediction], axis=1)
            next_predictions = np.array(next_predictions)
            future_dates = [pd.to_datetime(dates[-1]) + pd.DateOffset(days=i + 1) for i in range(num_of_prediction)]
            ensemble_future_predictions = np.array([scaler.inverse_transform(next_predictions[i]) for i in range(num_of_prediction)])


            # Calculate mean, P10, and P90 of predictions
            mean_predictions = ensemble_future_predictions[:, :, 0].mean(axis=1)
            P50 = np.percentile(ensemble_future_predictions[:, :, 0], 50, axis=1)
            P10 = np.percentile(ensemble_future_predictions[:, :, 0], 10, axis=1)
            P90 = np.percentile(ensemble_future_predictions[:, :, 0], 90, axis=1)

            # Create the plot
            plt.figure(figsize=(10, 6))

            # Plot historical EUA prices
            plt.plot(dates, original_EUA, label='Historical EUA Price', color='blue')

            # Plot all realizations
            for realization in ensemble_future_predictions[:, :, 0].T:
                plt.plot(future_dates, realization, color='gray', alpha=0.3)

            # Plot P10 and P90 percentile predictions
            plt.plot(future_dates, P10, label='P10 & P90', color='green', linestyle='-')
            plt.plot(future_dates, P90, color='green', linestyle='-')
            # Plot mean of future predictions
            plt.plot(future_dates, P50, label='Median of Predictions', color='red')


            # Customize the plot
            plt.title('EUA Price Prediction for the Next 24 Months')
            plt.xlabel('Date')
            plt.ylabel('EUA Price')
            plt.legend(loc='upper left')
            plt.grid(True)
            plt.savefig(f"{folder_name}/{modeltype}_timeplot_ts_{sequence_length}_factor_{str(factor).replace('.','_')}.pdf")


        record = {}
        with open(f'{folder_name}/{modeltype}_record_ts_{sequence_length}.txt', 'w') as f:
            # report metrics
            for i, feature in enumerate(predictors_lst):
                r2_train = r2_score(ground_truth_train[:,i],train_predictions_rescaled[:,i])
                r2_test = r2_score(ground_truth_train[:,i],train_predictions_rescaled[:,i])
                mse_train = mean_squared_error(ground_truth_train[:,i],train_predictions_rescaled[:,i])
                mse_test  = mean_squared_error(ground_truth_train[:,i],train_predictions_rescaled[:,i])
                mae_train = mean_absolute_error(ground_truth_train[:,i],train_predictions_rescaled[:,i])
                mae_test  = mean_absolute_error(ground_truth_train[:,i],train_predictions_rescaled[:,i])
                f.write(f'{feature}\n')
                f.write(f'r2(train): {r2_train}\n')
                f.write(f'r2(test): {r2_test}\n')
                f.write(f'mse(train): {mse_train}\n')
                f.write(f'mse(test): {mse_test}\n')
                f.write(f'mae(train): {mae_train}\n')
                f.write(f'mae(test): {mae_test}\n')
                f.write('---------------------------\n')
                record['feature'] = {"r2_train":r2_train, 
                                    "r2_test": r2_test,
                                    "mse_train": mse_train, 
                                    "mse_test": mse_test, 
                                    "mae_train": mae_train, 
                                    "mae_test": mae_test,}
        # save metric as dictionary
        with open(f'{folder_name}/{modeltype}_record_ts_{sequence_length}.pkl', 'wb') as f:
            pickle.dump(record, f)
        # Save the best model
        with open(f'{folder_name}/{modeltype}_bestmodel_ts_{sequence_length}.pkl', 'wb') as f:
            pickle.dump(best_model, f)
        with open(f'{folder_name}/{modeltype}_bestmodel_params_ts_{sequence_length}.pkl', 'wb') as f:
            pickle.dump(best_params, f)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   3.4s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   3.5s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   3.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   3.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   4.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   4.3s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total t

In [ ]:
sklearn_models.items()

dict_items([('rf', {'model': RandomForestRegressor(), 'param_grid': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}}), ('knn', {'model': KNeighborsRegressor(), 'param_grid': {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'p': [1, 2]}}), ('dt', {'model': DecisionTreeRegressor(), 'param_grid': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': [None, 'auto', 'sqrt', 'log2']}}), ('lasso', {'model': Lasso(), 'param_grid': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0], 'fit_intercept': [True, False], 'max_iter': [1000, 2000, 3000]}}), ('ridge', {'model': Ridge(), 'param_grid': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_intercept': [True, False], 'solver': ['auto', 'svd', 'cholesky', 'sparse_cg', 'saga'], 'max_iter': [1000, 2000, 3000]}}), ('gb', {'model': GradientBoostingRegressor(), 'param_grid': 

In [ ]:
    modeltype = 'rf'
    with open(f'sklearn_models/{modeltype}_record_ts_{sequence_length}.txt', 'w') as f:
        # report metrics
        for i, feature in enumerate(predictors_lst):
            r2_train = r2_score(ground_truth_train[:,i],train_predictions_rescaled[:,i])
            r2_test = r2_score(ground_truth_train[:,i],train_predictions_rescaled[:,i])

            mse_train = mean_squared_error(ground_truth_train[:,i],train_predictions_rescaled[:,i])
            mse_test = mean_squared_error(ground_truth_train[:,i],train_predictions_rescaled[:,i])

            mae_train = mean_absolute_error(ground_truth_train[:,i],train_predictions_rescaled[:,i])
            mae_test = mean_absolute_error(ground_truth_train[:,i],train_predictions_rescaled[:,i])
            f.write(f'{feature}\n')
            f.write(f'r2(train): {r2_train}\n')
            f.write(f'r2(test): {r2_test}\n')
            f.write(f'mse(train): {mse_train}\n')
            f.write(f'mse(test): {mse_test}\n')
            f.write(f'mae(train): {mae_train}\n')
            f.write(f'mae(test): {mae_test}\n')
            f.write('---------------------------\n')